# DATA EDA

### Imports

In [31]:
#Imports
import pandas as pd
import numpy as np

### Dataframes Imports 

In [32]:
#importing 1st transactions
transactions_0_df= pd.read_csv("../raw_data/transactions.csv")
#Importing Transactions data
transactions_df= pd.read_csv("../raw_data/transactions_v2.csv")
#Importing user logs data
user_logs_df = pd.read_csv("../raw_data/user_logs_v2.csv")
#importing members data
members_df = pd.read_csv("../raw_data/members_v3.csv")
# import first training dataset
train_0_df= pd.read_csv("../raw_data/train.csv")
# import training dataset
train_df = pd.read_csv("../raw_data/train_v2.csv")


### Building trendline

In [33]:
#concat vertically
transactions_df_concat = pd.concat([transactions_0_df, transactions_df], axis=0)

#drop duplicates
transactions_df_global = transactions_df_concat.drop_duplicates(keep="first")

In [34]:
#concat vertically
train_df_concat = pd.concat([train_0_df, train_df], axis=0)
train_df_concat

#drop duplicates
train_df_global = train_df_concat.drop_duplicates(keep="first")


In [35]:
#merge with transaction data
transactions_train_df = pd.merge(train_df_global, transactions_df_global, on='msno', how='left')
transactions_train_df

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,38,7,0,0,0,20161031,20161107,0
1,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,38,30,149,149,0,20170107,20170206,0
2,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,39,30,149,149,1,20161031,20161221,0
3,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,39,30,149,149,1,20160715,20160821,0
4,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,39,30,149,149,1,20160831,20161021,0
...,...,...,...,...,...,...,...,...,...,...
18393086,Zkuhc7S+hhMgLkKft6lfrsyi2XHDo6WH7QDdXlringE=,0,40,30,149,149,1,20151127,20151231,0
18393087,Zkuhc7S+hhMgLkKft6lfrsyi2XHDo6WH7QDdXlringE=,0,40,30,149,149,1,20160527,20160701,0
18393088,Zkuhc7S+hhMgLkKft6lfrsyi2XHDo6WH7QDdXlringE=,0,40,30,149,149,1,20160227,20160331,0
18393089,Zkuhc7S+hhMgLkKft6lfrsyi2XHDo6WH7QDdXlringE=,0,40,30,149,149,1,20161027,20161201,0


In [36]:
#convert to datetime
transactions_train_df.loc[:,'transaction_date'] = pd.to_datetime(transactions_train_df['transaction_date'], format='%Y%m%d')
transactions_train_df.loc[:,'membership_expire_date'] = pd.to_datetime(transactions_train_df['membership_expire_date'], format='%Y%m%d')

In [37]:
# Sort the DataFrame by 'msno' and 'transaction_date'
transactions_train_df = transactions_train_df.sort_values(by=['msno', 'transaction_date'], ascending=[True, True],)

In [38]:
#compute previous period churns
transactions_train_df['transaction_date_-1'] = transactions_train_df.groupby('msno')['transaction_date'].shift(+1)
transactions_train_df['membership_expire_date_-1'] = transactions_train_df.groupby('msno')['membership_expire_date'].shift(+1)
transactions_train_df['period_0'] = transactions_train_df['membership_expire_date_-1']-transactions_train_df['transaction_date']
transactions_train_df['period_0_churn'] = np.where(transactions_train_df['period_0'] <= pd.Timedelta(days=-30), 1, np.where(transactions_train_df['period_0'].isna(), np.nan, 0))

transactions_train_df['transaction_date_-2'] = transactions_train_df.groupby('msno')['transaction_date'].shift(+2)
transactions_train_df['membership_expire_date_-2'] = transactions_train_df.groupby('msno')['membership_expire_date'].shift(+2)
transactions_train_df['period_-1'] = transactions_train_df['membership_expire_date_-2']-transactions_train_df['transaction_date_-1']
transactions_train_df['period_-1_churn'] = np.where(transactions_train_df['period_-1'] <= pd.Timedelta(days=-30), 1, np.where(transactions_train_df['period_-1'].isna(), np.nan, 0))


transactions_train_df['transaction_date_-3'] = transactions_train_df.groupby('msno')['transaction_date'].shift(+3)
transactions_train_df['membership_expire_date_-3'] = transactions_train_df.groupby('msno')['membership_expire_date'].shift(+3)
transactions_train_df['period_-2'] = transactions_train_df['membership_expire_date_-3']-transactions_train_df['transaction_date_-2']
transactions_train_df['period_-2_churn'] = np.where(transactions_train_df['period_-2'] <= pd.Timedelta(days=-30), 1, np.where(transactions_train_df['period_-2'].isna(), np.nan, 0))


transactions_train_df['transaction_date_-4'] = transactions_train_df.groupby('msno')['transaction_date'].shift(+4)
transactions_train_df['membership_expire_date_-4'] = transactions_train_df.groupby('msno')['membership_expire_date'].shift(+4)
transactions_train_df['period_-3'] = transactions_train_df['membership_expire_date_-4']-transactions_train_df['transaction_date_-3']
transactions_train_df['period_-3_churn'] = np.where(transactions_train_df['period_-3'] <= pd.Timedelta(days=-30), 1, np.where(transactions_train_df['period_-3'].isna(), np.nan, 0))


transactions_train_df['transaction_date_-5'] = transactions_train_df.groupby('msno')['transaction_date'].shift(+5)
transactions_train_df['membership_expire_date_-5'] = transactions_train_df.groupby('msno')['membership_expire_date'].shift(+5)
transactions_train_df['period_-4'] = transactions_train_df['membership_expire_date_-5']-transactions_train_df['transaction_date_-4']
transactions_train_df['period_-4_churn'] = np.where(transactions_train_df['period_-4'] <= pd.Timedelta(days=-30), 1, np.where(transactions_train_df['period_-4'].isna(), np.nan, 0))


transactions_train_df['transaction_date_-6'] = transactions_train_df.groupby('msno')['transaction_date'].shift(+6)
transactions_train_df['membership_expire_date_-6'] = transactions_train_df.groupby('msno')['membership_expire_date'].shift(+6)
transactions_train_df['period_-5'] = transactions_train_df['membership_expire_date_-6']-transactions_train_df['transaction_date_-5']
transactions_train_df['period_-5_churn'] = np.where(transactions_train_df['period_-5'] <= pd.Timedelta(days=-30), 1, np.where(transactions_train_df['period_-5'].isna(), np.nan, 0))




In [39]:
#only keeping rows with the latest transactions 
latest_transaction_indexes = transactions_train_df.groupby('msno')['transaction_date'].idxmax()
latest_transactions_with_churn = transactions_train_df.loc[latest_transaction_indexes]

In [40]:
# removing unnecessary columns
trendline_df = latest_transactions_with_churn.drop(axis=1, columns=[
    'transaction_date_-1',
    'membership_expire_date_-1',
    'period_0',
    'transaction_date_-2',
    'membership_expire_date_-2',
    'period_-1',
    'transaction_date_-3',
    'membership_expire_date_-3',
    'period_-2',
    'transaction_date_-4',
    'membership_expire_date_-4',
    'period_-3',
    'transaction_date_-5',
    'membership_expire_date_-5',
    'period_-4',
    'transaction_date_-6',
    'membership_expire_date_-6',
    'period_-5',
    'is_churn',
    'payment_method_id',
    'payment_plan_days',
    'plan_list_price',
    'actual_amount_paid',
    'is_auto_renew',
    'transaction_date',
    'membership_expire_date',
    'is_cancel'
])
                                                                        

### Transactions Data

In [41]:
###### converting transaction_date and membership_date to datetime object
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'], format='%Y%m%d')
transactions_df['membership_expire_date'] = pd.to_datetime(transactions_df['membership_expire_date'], format='%Y%m%d')

# removing duplicates and leaving only the latest transaction date
transactions_df_lt = transactions_df.loc[transactions_df.groupby('msno').transaction_date.idxmax()]

# creating remaining plan duration column
transactions_df_lt['remaining_plan_duration'] = transactions_df_lt['membership_expire_date'] - transactions_df_lt['transaction_date']

# creating discount column
transactions_df_lt['is_discount'] = transactions_df_lt.apply(lambda x: '0' if (x['actual_amount_paid'] -x['plan_list_price'])>=0 else '1', axis=1)

### User logs Data

In [42]:
# Converting date into datetime object
user_logs_df['date'] = pd.to_datetime(user_logs_df['date'], format='%Y%m%d')

#create new dataframe from transactions table with only msno and latest transaction date
latest_transactions_per_msno = transactions_df_lt[['msno', 'transaction_date']]

#Merge user logs with latest transaction date
merged_df = user_logs_df.merge(latest_transactions_per_msno, on='msno', how='left')

# drop msno's where transaction date is NaN
merged_df = merged_df.dropna(subset=['transaction_date'])

#removing rows where the user log data is before the last transaction date
user_logs_atd= merged_df.loc[merged_df['date']>=merged_df['transaction_date']]

# removing data column
user_logs_atd = user_logs_atd.drop(columns=['date','transaction_date'])

#groupby msno and summing all values
user_logs_atd = user_logs_atd.groupby('msno').sum()

### Merging

In [43]:
# Merging transactions, user logs and members data to Train dataframe
train_df = train_df.merge(transactions_df_lt, on='msno', how='left')
train_df = train_df.merge(user_logs_atd, on='msno', how='left')
train_df = train_df.merge(members_df, on='msno', how='left')
train_df = train_df.merge(trendline_df, on='msno', how='left')

### Feature engineering

In [44]:
# Drop Gender column
train_df = train_df.drop(['gender'],axis=1)

In [45]:
# Drop NaN
train_df = train_df.dropna()

In [46]:
# Changing bd outliers to NaN
train_df['bd'] = train_df['bd'].apply(lambda x: np.nan if x <14 or x > 75 else x)

In [47]:
# Converting remaining plan duration to int
train_df['remaining_plan_duration'] = train_df['remaining_plan_duration'].dt.days

In [48]:
# Converting total seconds to hours
train_df['total_secs']= round(train_df['total_secs']/3600,2)

In [49]:
# Creating average usage (hours) per day from latest transaction
train_df['usage_from_ltd'] = round(train_df['total_secs']/train_df['remaining_plan_duration'],2)
# replacing Nan with 0 
train_df['usage_from_ltd'].replace([np.nan], 0, inplace=True)
# replacing inf values with the totalsecs usage.
train_df['usage_from_ltd'] = np.where(train_df['usage_from_ltd'] == np.inf, train_df['total_secs'],train_df['usage_from_ltd'])

In [50]:
# converting registration time to datetime object
train_df['registration_init_time'] = pd.to_datetime(train_df['registration_init_time'], format='%Y%m%d')

In [51]:
# creating discount percentage column
train_df['discount_percentage'] = round((train_df['plan_list_price'] - train_df['actual_amount_paid'])/train_df['plan_list_price'],2)

train_df['discount_percentage'].replace([np.nan,-0.01,0.01], 0, inplace=True)

### Cyclic Encoding

In [52]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [53]:
# cyclic encoding transaction date
#year
train_df['last_transaction_year'] = train_df['transaction_date'].dt.year

#month
train_df['last_transaction_month'] = train_df['transaction_date'].dt.month
train_df = encode(train_df, 'last_transaction_month', 12)

#day
train_df['last_transaction_day'] = train_df['transaction_date'].dt.day
train_df= encode(train_df, 'last_transaction_day', 31)

train_df =train_df.drop(['last_transaction_month', 'last_transaction_day'],axis=1)

In [54]:
# cyclic encoding membership expire date
#year
train_df['expire_year'] = train_df['membership_expire_date'].dt.year

#month
train_df['expire_month'] = train_df['membership_expire_date'].dt.month
train_df = encode(train_df, 'expire_month', 12)

#day
train_df['expire_day'] = train_df['membership_expire_date'].dt.day
train_df= encode(train_df, 'expire_day', 31)

# droping month and day columns
train_df =train_df.drop(['expire_month', 'expire_day'],axis=1)

In [55]:
# cyclic encoding registration date
#year
train_df['registration_year'] = train_df['registration_init_time'].dt.year

#month
train_df['registration_month'] = train_df['registration_init_time'].dt.month
train_df = encode(train_df, 'registration_month', 12)

#day
train_df['registration_day'] = train_df['registration_init_time'].dt.day
train_df= encode(train_df, 'registration_day', 31)

# droping month and day columns
train_df =train_df.drop(['registration_month', 'registration_day'],axis=1)


In [56]:
# droping date columns to avoid data leakege
train_df =train_df.drop(['registration_init_time', 'membership_expire_date', 'transaction_date'],axis=1)

In [57]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 535129 entries, 2 to 970956
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   msno                        535129 non-null  object 
 1   is_churn                    535129 non-null  int64  
 2   payment_method_id           535129 non-null  float64
 3   payment_plan_days           535129 non-null  float64
 4   plan_list_price             535129 non-null  float64
 5   actual_amount_paid          535129 non-null  float64
 6   is_auto_renew               535129 non-null  float64
 7   is_cancel                   535129 non-null  float64
 8   remaining_plan_duration     535129 non-null  int64  
 9   is_discount                 535129 non-null  object 
 10  num_25                      535129 non-null  float64
 11  num_50                      535129 non-null  float64
 12  num_75                      535129 non-null  float64
 13  num_985            

In [58]:
train_df.value_counts('is_churn')

is_churn
0    507132
1     27997
Name: count, dtype: int64

In [59]:
df_no_churn = train_df[train_df['is_churn'] == 0]
df_churn = train_df[train_df['is_churn'] == 1]
df_no_churn = df_no_churn.sample(27997, random_state=42)
train_underbalancing = pd.concat([df_no_churn, df_churn], axis=0)
train_underbalancing.to_csv('churn_df_underbalanced.csv')